In [1]:
from System import system
import numpy as np
import numpy
import sympy as sp
from steady_state_values import steady_state
# sp.init_printing()

In [2]:
t = sp.Symbol('t')
s = sp.Symbol('s')
theta = sp.Symbol('theta')

Assumptions:
Bad mixing is approximated by 3 jacket system. We will assume that the bottom 3rd of the tank is always full as this is already well outside the operating limits. therefore h1 = L/3 , h2 = L/3 (

Since state space models cannot handle dead-time, Cc_measured is not considered as an output but instead Cc is. Deadtime does not affect the poles or zeros of a system and once a transfer function representation is obtained, one can simply add an e^(-theta*s) term to get to Cc_measured from Cc.

In [3]:
# Parameters (do not change these unless specifically asked)\
pi = numpy.pi
Ko = 2.941e8  # m^3/(s*kmol)
Ea = 75360  # KJ/kmol
R = 8.313  # KJ/(kmol K)
rho_a = 801.2  # kg/m^3
rho_b = 966.2  # kg/m^3
rho_c = 1004   # kg/m^3
rho_solvent = 764.6  # kg/m^3
rhocw = 966.2  # kg/m^3 Water density
MM_A = 58.1  # kg/kmol
MM_B = 18  # kg/kmol
MM_C = 76.09  # kg/kmol
Cv1 = 3.089e-3  # m^2
alpha1 = 2
delta_Pv1 = 703.6  # Pa
delta_Pv3 = 1000  #Pa
alpha3 = 2
Cv2 = 1.33e-3  # m^2
Cv3 = (2.360e-3/(2**(-0.5)))/(numpy.sqrt(delta_Pv3/966.2))  #m^2
alpha2 = 2
g = 9.807  # m/s^2
tau_v1 = 36
zeta_v1 = 0.35
Kv1 = 1/80  # 1/Kpa
tau_v2 = 36
tau_v3 = 36
Kv2 = 1/80  # 1/Kpa
Kv3 = 1/80
Cp1 = 2.522  # KJ/kg degC
Cp2 = 4.187  # KJ/kg degC
Cp3 = 2.531  # KJ/kg degC
Cpcw = 4.187  # Kj/kg degC
delta_H = -84660  # Kj/kmol A
Uj = 4.750  # kW/(m^2 degC)
D = 1  # m
U_air = 44.5/1000  # kW/(m^2 degC)
L = 1.5*D  # m
A = (pi/4)*(D**2)  # m^2
To = 21.1  # degC
theta = 90  # s
Vj = ((pi/4)*(D*1.1)**2 - A)*L/3


# assumed constant
Cbo = 50 # kmol/m3
Tcwo = 29.4 # degC
Tao = 24 # degC


In [8]:


## equations

var = ['Ca',
       'Cb',
       'Cc',
       'Na',
       'K',
       'V',
       'p1',
       'p2',
       'p3',
       'rao',
       'rbo',
       'ra',
       'rb',
       'rc',
       'rsolvent',
       'F2',
       'F3',
       'Fcw',
       'del_PV2',
       'xv1',
       'xv2',
       'xv3',
       'T',
       'Tcw',
       'Qj',
       'Qair',
       'H',
       'Cao',
       'F1',
       'Ps1',
       'Ps2',
       'Ps3',
       'Tbo',
       'Fcw',
       'Tcw',
       'dVCa_dt',
        'dVCb_dt',
      'dVCc_dt',
      'dVp3_dt',
      'dxv1_dt',
      'dxv2_dt',
      'dxv3_dt',
      'z',
      'dz_dt',
      'dxv2_dt',
      'dxv3_dt',
      'dp3VT_dt',
      'dFcwT_dt']

[Ca,Cb,Cc,Na,K,V,p1,p2,p3,rao,rbo,ra,rb,rc,rsolvent,F2,F3,Fcw,del_PV2,xv1,xv2,xv3,
 T,Tcw,Qj,Qair,H,Cao,F1,Ps1,Ps2,Ps3,Tbo,Fcw,Tcw,dVCa_dt,dVCb_dt,dVCc_dt,dVp3_dt,dxv1_dt,
 dxv2_dt,dxv3_dt,z,dz_dt,dxv2_dt,dxv3_dt,dp3VT_dt,dFcwT_dt] = sp.symbols(var)

dstates = [dVCa_dt,dVCb_dt,dVCc_dt,dVp3_dt,dxv1_dt,
 dxv2_dt,dxv3_dt,z,dz_dt,dxv2_dt,dxv3_dt,dp3VT_dt,dFcwT_dt]

ss_ders = np.zeros(len(dstates))
ss_subs= zip(dstates,ss_ders)

states= [V*Ca,V*Cb,V*Cc,V*p3,xv1,xv2,xv3,z,xv2,xv3,p3*V*T,Fcw*Tcw]

variables =  [F2, F3,Fcw, Ca, Cb, Cc, Na,K,V,xv1,z,xv2,xv3, p1,p2,p3,T,Qj,Tcw,Qair,rao,
              rbo,ra,rb,rc,rsolvent,H,del_PV2]

outputs = [F2, F3,Fcw, Ca, Cb, Cc, Na,K,V, p1,p2,p3,T,Qj,Tcw,Qair,rao,
              rbo,ra,rb,rc,rsolvent,H,del_PV2]

exogenous_inputs = [Ps1,Ps2,Ps3,Cao,Tbo,F1]

LHS = [F1*Cao,
       F2*Cbo,
       Na*V,
       Na,
       K,
       p1*F1+p2*F2,
       1/p1,
      1/p2,
      1/p3,
      rao,
      rbo,
      ra,
      rb,
      rc,
      rsolvent,
      F2,
      Fcw,
      F3,
      del_PV2,
      tau_v2**2 * dz_dt + 2* zeta_v1*tau_v1*z + xv1,
      z,
      tau_v2*dxv2_dt + xv2,
      tau_v3*dxv3_dt + xv3,
      p1*F1*Cp1*Tao+p2*F2*Cp2*Tbo - delta_H*ra,
      dFcwT_dt,
      Qj,
      Qair,
      H]

RHS = [F3*Ca + Na*V + dVCa_dt,
       F3*Cb + Na*V + dVCb_dt,
       F3*Cc + dVCc_dt,
       K*Ca*Cb,
       Ko*sp.exp(-Ea/(R*T)),
       p3*F3 + dVp3_dt,
       rao/rho_a + (1-rao)/rho_solvent,
       rbo/rho_b + (1-rbo)/rho_solvent,
       ra/rho_a + rb/rho_b + rc/rho_c + rsolvent/rho_solvent,
       Cao*MM_A/p1,
       Cbo*MM_B/p2,
       Ca*MM_A/p3,
       Cb*MM_B/p3,
       Cc*MM_C/p3,
       1 - (ra+rb+rc),
       Cv1*alpha1**(xv1-1)*sp.sqrt(delta_Pv1/p2),
       Cv3*alpha3**(xv3-1)*sp.sqrt(delta_Pv3/rhocw),
       Cv2*alpha2**(xv2-1)*sp.sqrt(del_PV2/p3),
       p3*g*H,
       Kv1*Ps1,
       dxv1_dt,
       Kv2*Ps2,
       Kv3*Ps3,
       p3*F3*Cp3*T + Qj + Cp3*dp3VT_dt,
       Fcw*rhocw*(Tcwo-Tcw) + Qj - Qair,
       Uj*(pi*D)*(H*(T-Tcw)),
       U_air*(pi*D)*(L-H)*(T-Tcw),  
       V/A]


In [9]:
# check dof

len(variables) - len(LHS)

0

In [ ]:
# getting steady state
RhS = np.array([f.subs(zip(dstates,ss_ders)) for f in RHS])
LhS = np.array([f.subs(zip(dstates,ss_ders)) for f in LHS])
prob = RhS - LhS
sol = sp.solve(prob,variables)
ss = solution
ss


In [ ]:
# linearizing

def lin(f,var,ss):  # linearizes for all variables that we have defined. will not touch derrivatives(Rightfully so)
    ret = 0
    for i, v in enumerate(var):
        df = sp.diff(f,v)
        for j, k in enumerate(var):
            if j != i:
                df.subs(k,ss[j])
        df *= (v - ss[i])
        ret += df
    return ret